In [1]:
import os

import pandas as pd
from sklearn.externals import joblib

from keras.models import load_model

Using TensorFlow backend.


In [2]:
import numpy as np

from sklearn.metrics import accuracy_score

In [3]:
old_csv = [x for x in os.listdir("data") if x.endswith('edited.csv')]

In [4]:
old_csv

['EOD-MSFT-edited.csv',
 'BTC-USD-edited.csv',
 'GOOG-edited.csv',
 'EOD-DIS-edited.csv',
 'EOD-MMM-edited.csv',
 'NFLX-edited.csv',
 'AMZN-edited.csv',
 'AAPL-edited.csv',
 'EOD-V-edited.csv',
 'TSLA-edited.csv',
 'FB-edited.csv']

In [5]:
old_csv_filenames = []

for filename in old_csv:
    print('data/%s'%filename)
    old_csv_filenames.append('data/%s'%filename)

data/EOD-MSFT-edited.csv
data/BTC-USD-edited.csv
data/GOOG-edited.csv
data/EOD-DIS-edited.csv
data/EOD-MMM-edited.csv
data/NFLX-edited.csv
data/AMZN-edited.csv
data/AAPL-edited.csv
data/EOD-V-edited.csv
data/TSLA-edited.csv
data/FB-edited.csv


In [6]:
clf = joblib.load('data/bitcoin-predictor.pkl')

In [7]:
for filename in old_csv_filenames:
    df = pd.read_csv(filename)
    
    X = df[['Value USD', 'Max 7', 'Min 7', 'Change', 'Mean Change 7', 'Drop 7', 'Up 7']].values
    y = df['Actual'].values
    
    score = clf.score(X, y)
    print('{0} score: {1}'.format(filename, score))

data/EOD-MSFT-edited.csv score: 0.9616562538282494
data/BTC-USD-edited.csv score: 0.9383468834688347
data/GOOG-edited.csv score: 0.9399031614924523
data/EOD-DIS-edited.csv score: 0.4582221598090156
data/EOD-MMM-edited.csv score: 0.9630354957160343
data/NFLX-edited.csv score: 0.9479754601226994
data/AMZN-edited.csv score: 0.9471613265879708
data/AAPL-edited.csv score: 0.9641388039236367
data/EOD-V-edited.csv score: 0.975095785440613
data/TSLA-edited.csv score: 0.9469548133595285
data/FB-edited.csv score: 0.9448364336112893


In [8]:
model = load_model('data/amzn-predictor-shuffled.hd5')

In [9]:
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df.as_matrix(result).astype(np.float32), dummies.as_matrix().astype(np.float32)
    else:
        # Regression
        return df.as_matrix(result).astype(np.float32), df.as_matrix([target]).astype(np.float32)

In [10]:
for filename in old_csv_filenames:
    df = pd.read_csv(filename)
    
    df.drop(['Date'], inplace=True, axis=1)
    X,y = to_xy(df,"Actual")
    
    pred = model.predict(X)
    predict_classes = np.argmax(pred,axis=1)
    y_arg = np.argmax(y, axis=1)
    
    score = accuracy_score(y_arg,predict_classes)
    print('{0} score: {1}'.format(filename, score))

/home/sam/code/envs/data/lib/python3.6/site-packages/ipykernel_launcher.py:14: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


data/EOD-MSFT-edited.csv score: 0.591694230062477
data/BTC-USD-edited.csv score: 0.9525745257452575
data/GOOG-edited.csv score: 0.4793506123611507
data/EOD-DIS-edited.csv score: 0.4684735289987361
data/EOD-MMM-edited.csv score: 0.48706650346797226
data/NFLX-edited.csv score: 0.5293251533742331
data/AMZN-edited.csv score: 0.4892261570170508
data/AAPL-edited.csv score: 0.711106423373062
data/EOD-V-edited.csv score: 0.5344827586206896
data/TSLA-edited.csv score: 0.48379174852652257
data/FB-edited.csv score: 0.4644002565747274
